In [1]:
#引入CutMix
from cutmix.cutmix import CutMix
from cutmix.utils import CutMixCrossEntropyLoss

In [2]:
#引入TTA
import ttach as tta

In [4]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from resnest.torch import resnest50
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision import transforms
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.model_selection import KFold
from PIL import Image
import os
import matplotlib.pyplot as plt
import torchvision.models as models
from tqdm import tqdm
import seaborn as sns

In [6]:
labels_dataframe = pd.read_csv('train.csv')
labels_dataframe.head(5)

,image,label
0,images/0.jpg,maclura_pomifera
1,images/1.jpg,maclura_pomifera
2,images/2.jpg,maclura_pomifera
3,images/3.jpg,maclura_pomifera
4,images/4.jpg,maclura_pomifera


In [8]:
leaves_labels = sorted(list(set(labels_dataframe['label'])))
n_classes = len(leaves_labels)
print(n_classes)
leaves_labels[:5]

176


['abies_concolor',
 'abies_nordmanniana',
 'acer_campestre',
 'acer_ginnala',
 'acer_griseum']

In [19]:
class_to_num = dict(zip(leaves_labels, range(n_classes)))
num_to_class = {v : k for k, v in class_to_num.items()}

In [ ]:
def barw(ax): 
    
    for p in ax.patches:
        val = p.get_width() #height of the bar
        x = p.get_x()+ p.get_width() # x- position 
        y = p.get_y() + p.get_height()/2 #y-position
        ax.annotate(round(val,2),(x,y))
        
#finding top leaves

plt.figure(figsize = (15,30))
ax0 =sns.countplot(y=labels_dataframe['label'],order=labels_dataframe['label'].value_counts().index)
barw(ax0)
plt.show()

In [13]:
# 继承pytorch的dataset
class TrainValidData(Dataset):
    def __init__(self, csv_path, file_path, resize_height=224, resize_width=224, transform=None):
        
        # 需要调整后的照片尺寸，我这里每张图片的大小尺寸不一致#
        self.resize_height = resize_height
        self.resize_width = resize_width
        self.file_path = file_path
        self.to_tensor = transforms.ToTensor() #将数据转换成tensor形式
        self.transform = transform

        # 利用pandas读取csv文件
        self.data_info = pd.read_csv(csv_path, header=None)  #header=None是去掉表头部分
        # 文件第一列包含图像文件名称
        self.image_arr = np.asarray(self.data_info.iloc[1:,0]) #self.data_info.iloc[1:,0]表示读取第一列，从第二行开始一直读取到最后一行
        # 第二列是图像的 label
        self.label_arr = np.asarray(self.data_info.iloc[1:,1])
        # 计算 length
        self.data_len = len(self.data_info.index) - 1

    def __getitem__(self, index):
        # 从 image_arr中得到索引对应的文件名
        single_image_name = self.image_arr[index]

        # 读取图像文件
        img_as_img = Image.open(self.file_path + single_image_name)
        
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor()
        ])
        img_as_img = transform(img_as_img)

        label = self.label_arr[index]
        number_label = class_to_num[label]

        return (img_as_img, number_label) 

    def __len__(self):
        return self.data_len

In [14]:
# 继承pytorch的dataset
class TestData(Dataset):
    def __init__(self, csv_path, file_path, resize_height=224, resize_width=224, transform = None):
        
        self.resize_height = resize_height
        self.resize_width = resize_width
        self.file_path = file_path
        self.transform = transform
        self.to_tensor = transforms.ToTensor() #将数据转换成tensor形式

        self.data_info = pd.read_csv(csv_path, header=None)  
        # 文件第一列包含图像文件名称
        self.image_arr = np.asarray(self.data_info.iloc[1:,0]) 
        # 计算 length
        self.data_len = len(self.data_info.index) - 1
        
    def __getitem__(self, index):
        # 从 image_arr中得到索引对应的文件名
        single_image_name = self.image_arr[index]

        # 读取图像文件
        img_as_img = Image.open(self.file_path + single_image_name)
       
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor()
        ])
        img_as_img = transform(img_as_img)


        return img_as_img

    def __len__(self):
        return self.data_len

In [15]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.08, 1.0), ratio=(3.0 / 4.0, 4.0 / 3.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])
val_test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

In [16]:
train_val_path = 'train.csv'
test_path = 'test.csv'

img_path = 'autodl-tmp/'

train_val_dataset = TrainValidData(train_val_path, img_path)
test_dataset = TestData(test_path, img_path, transform = val_test_transform)
print(train_val_dataset.data_info)
print(test_dataset.data_info)

                      0                        1
0                 image                    label
1          images/0.jpg         maclura_pomifera
2          images/1.jpg         maclura_pomifera
3          images/2.jpg         maclura_pomifera
4          images/3.jpg         maclura_pomifera
...                 ...                      ...
18349  images/18348.jpg          aesculus_glabra
18350  images/18349.jpg  liquidambar_styraciflua
18351  images/18350.jpg            cedrus_libani
18352  images/18351.jpg      prunus_pensylvanica
18353  images/18352.jpg          quercus_montana

[18354 rows x 2 columns]
                     0
0                image
1     images/18353.jpg
2     images/18354.jpg
3     images/18355.jpg
4     images/18356.jpg
...                ...
8796  images/27148.jpg
8797  images/27149.jpg
8798  images/27150.jpg
8799  images/27151.jpg
8800  images/27152.jpg

[8801 rows x 1 columns]


In [17]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        model = model
        for param in model.parameters():
            param.requires_grad = False

# ResNeSt模型,注意力机制模型
def resnest_model(num_classes, feature_extract = False):
    model_ft = resnest50(pretrained=True)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Sequential(nn.Linear(num_ftrs, num_classes))

    return model_ft

In [18]:
# 看一下是在cpu还是GPU上
def get_device():
    return 'cuda' if torch.cuda.is_available() else 'cpu'

device = get_device()
print(device)

cuda


In [17]:
k_folds = 5
num_epochs = 30
learning_rate = 1e-4
weight_decay = 1e-3
train_loss_function = CutMixCrossEntropyLoss(True)
valid_loss_function = nn.CrossEntropyLoss()
# For fold results
results = {}

torch.manual_seed(42)

kfold = KFold(n_splits=k_folds, shuffle=True)

In [ ]:
print('--------------------------------------')

for fold, (train_ids,valid_ids) in enumerate(kfold.split(train_val_dataset)):


  print(f'FOLD {fold}')
  print('--------------------------------------')

  
  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  valid_subsampler = torch.utils.data.SubsetRandomSampler(valid_ids)

  
  trainloader = torch.utils.data.DataLoader(
                      CutMix(TrainValidData(train_val_path, img_path, transform = train_transform), num_class=176, beta=1.0, prob=0.5, num_mix=2), 
                      batch_size=32, sampler=train_subsampler, num_workers=0)
  validloader = torch.utils.data.DataLoader(
                      TrainValidData(train_val_path, img_path, transform = val_test_transform),
                      batch_size=32, sampler=valid_subsampler, num_workers=0)
  
  model = resnest_model(176)
  model = model.to(device)
  model.device = device
  
  optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate,weight_decay= weight_decay)
  scheduler = CosineAnnealingLR(optimizer,T_max=10)

  # Run the training loop for defined number of epochs
  for epoch in range(0,num_epochs):
    model.train()
    # Print epoch
    print(f'Starting epoch {epoch+1}')
    # These are used to record information in training
    train_losses = []
    train_accs = []
    # Iterate the training set by batches
    for batch in tqdm(trainloader):
      # Move images and labels to GPU
      imgs, labels = batch
      imgs = imgs.to(device)
      labels = labels.to(device)
      # Forward the data
      logits = model(imgs)
      # Calculate loss
      loss = train_loss_function(logits,labels)
      # Clear gradients in previous step
      optimizer.zero_grad()
      # Compute gradients for parameters
      loss.backward()
      # Update the parameters with computed gradients
      optimizer.step()
      # Compute the accuracy for current batch.
      # acc = (logits.argmax(dim=-1) == labels).float().mean()
      # Record the loss and accuracy.
      train_losses.append(loss.item())
      # train_accs.append(acc)
    print("第%d个epoch的学习率：%f" % (epoch+1,optimizer.param_groups[0]['lr']))
    scheduler.step()
    # The average loss and accuracy of the training set is the average of the recorded values.
    train_loss = np.sum(train_losses) / len(train_losses)
    # train_acc = np.sum(train_accs) / len(train_accs)
    # Print the information.
    # print(f"[ Train | {epoch + 1:03d}/{num_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")
    print(f"[ Train | {epoch + 1:03d}/{num_epochs:03d} ] loss = {train_loss:.5f}")

  # Train process (all epochs) is complete
  print('Training process has finished. Saving trained model.')
  print('Starting validation')

  # Saving the model
  print('saving model with loss {:.3f}'.format(train_loss))
  save_path = f'./model-fold-{fold}.pth'
  torch.save(model.state_dict(),save_path)
  # Start Validation
  model.eval()
  valid_losses = []
  valid_accs = []
  with torch.no_grad():
    for batch in tqdm(validloader):
      imgs, labels = batch
      # No gradient in validation
      logits = model(imgs.to(device))
      loss = valid_loss_function(logits,labels.to(device))
      acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
      # Record loss and accuracy
      valid_losses.append(loss.item())        
      valid_accs.append(acc.cpu().numpy())
    # The average loss and accuracy
    valid_loss = np.sum(valid_losses)/len(valid_losses)
    valid_acc = np.sum(valid_accs)/len(valid_accs)
    print(f"[ Valid | {epoch + 1:03d}/{num_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")
    print('Accuracy for fold %d: %d' % (fold, valid_acc))
    print('--------------------------------------')
    results[fold] = valid_acc
# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
total_summation = 0.0
for key, value in results.items():
  print(f'Fold {key}: {value} ')
  total_summation += value
print(f'Average: {total_summation/len(results.items())} ')

In [19]:
testloader = torch.utils.data.DataLoader(
                      TestData(test_path, img_path, transform = val_test_transform),
                      batch_size=32, num_workers=0)

In [20]:
## predict
model = resnest_model(176)

# create model and load weights from checkpoint
model = model.to(device)
# load the all folds
for test_fold in range(k_folds):
  model_path = f'./model-fold-{test_fold}.pth'
  saveFileName = f'./submission-fold-{test_fold}.csv'
  model.load_state_dict(torch.load(model_path))

  # Make sure the model is in eval mode.
  # Some modules like Dropout or BatchNorm affect if the model is in training mode.
  model.eval()
  tta_model = tta.ClassificationTTAWrapper(model, tta.aliases.five_crop_transform(200,200)) # Test-Time Augmentation

  # Initialize a list to store the predictions.
  predictions = []
  # Iterate the testing set by batches.
  for batch in tqdm(testloader):
      
      imgs = batch
      with torch.no_grad():
          logits = tta_model(imgs.to(device))
      
      # Take the class with greatest logit as prediction and record it.
      predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())

  preds = []
  for i in predictions:
      preds.append(num_to_class[i])

  test_data = pd.read_csv(test_path)
  test_data['label'] = pd.Series(preds)
  submission = pd.concat([test_data['image'], test_data['label']], axis=1)
  submission.to_csv(saveFileName, index=False)
  print("ResNeSt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!")

100%|██████████| 275/275 [00:38<00:00,  7.09it/s]


ResNeSt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


100%|██████████| 275/275 [00:38<00:00,  7.15it/s]


ResNeSt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


100%|██████████| 275/275 [00:40<00:00,  6.80it/s]


ResNeSt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


100%|██████████| 275/275 [00:40<00:00,  6.78it/s]


ResNeSt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


100%|██████████| 275/275 [00:39<00:00,  6.96it/s]

ResNeSt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [21]:
# 读取5折交叉验证的结果
df0 = pd.read_csv('./submission-fold-0.csv')
df1 = pd.read_csv('./submission-fold-1.csv')
df2 = pd.read_csv('./submission-fold-2.csv')
df3 = pd.read_csv('./submission-fold-3.csv')
df4 = pd.read_csv('./submission-fold-4.csv')

In [22]:
# 往第0折结果里添加数字化标签列
list_num_label0 = []
for i in df0['label']:
  list_num_label0.append(class_to_num[i])
df0['num_label0']=list_num_label0
df0.head()

,image,label,num_label0
0,images/18353.jpg,asimina_triloba,22
1,images/18354.jpg,betula_nigra,26
2,images/18355.jpg,platanus_acerifolia,120
3,images/18356.jpg,pinus_bungeana,102
4,images/18357.jpg,platanus_acerifolia,120


In [23]:
# 往第1折结果里添加数字化标签列
list_num_label1 = []
for i in df1['label']:
  list_num_label1.append(class_to_num[i])
df1['num_label1']=list_num_label1
df1.head()

,image,label,num_label1
0,images/18353.jpg,asimina_triloba,22
1,images/18354.jpg,betula_nigra,26
2,images/18355.jpg,platanus_acerifolia,120
3,images/18356.jpg,pinus_bungeana,102
4,images/18357.jpg,platanus_acerifolia,120


In [24]:
# 往第2折结果里添加数字化标签列
list_num_label2 = []
for i in df2['label']:
  list_num_label2.append(class_to_num[i])
df2['num_label2']=list_num_label2
df2.head()

,image,label,num_label2
0,images/18353.jpg,asimina_triloba,22
1,images/18354.jpg,betula_nigra,26
2,images/18355.jpg,platanus_acerifolia,120
3,images/18356.jpg,pinus_bungeana,102
4,images/18357.jpg,platanus_acerifolia,120


In [25]:
# 往第3折结果里添加数字化标签列
list_num_label3 = []
for i in df3['label']:
  list_num_label3.append(class_to_num[i])
df3['num_label3']=list_num_label3
df3.head()

,image,label,num_label3
0,images/18353.jpg,asimina_triloba,22
1,images/18354.jpg,betula_nigra,26
2,images/18355.jpg,platanus_acerifolia,120
3,images/18356.jpg,pinus_bungeana,102
4,images/18357.jpg,platanus_acerifolia,120


In [26]:
# 往第4折结果里添加数字化标签列
list_num_label4 = []
for i in df4['label']:
  list_num_label4.append(class_to_num[i])
df4['num_label4']=list_num_label4
df4.head()

,image,label,num_label4
0,images/18353.jpg,asimina_triloba,22
1,images/18354.jpg,betula_nigra,26
2,images/18355.jpg,platanus_acerifolia,120
3,images/18356.jpg,pinus_bungeana,102
4,images/18357.jpg,platanus_acerifolia,120


In [27]:
# 准备整合5折的结果到同一个DataFrame
df_all = df0.copy()
df_all.drop(['label'],axis=1,inplace=True)
df_all.head()

,image,num_label0
0,images/18353.jpg,22
1,images/18354.jpg,26
2,images/18355.jpg,120
3,images/18356.jpg,102
4,images/18357.jpg,120


In [28]:
# 整合5折的数字化标签结果到同一个DataFrame
df_all['num_label1']=list_num_label1
df_all['num_label2']=list_num_label2
df_all['num_label3']=list_num_label3
df_all['num_label4']=list_num_label4
df_all.head()

,image,num_label0,num_label1,num_label2,num_label3,num_label4
0,images/18353.jpg,22,22,22,22,22
1,images/18354.jpg,26,26,26,26,26
2,images/18355.jpg,120,120,120,120,120
3,images/18356.jpg,102,102,102,102,102
4,images/18357.jpg,120,120,120,120,120


In [29]:
# 对df_all进行转置，方便求众数
df_all_transpose = df_all.copy().drop(['image'],axis=1).transpose()
df_all_transpose.head()

,0,1,2,3,4,5,6,7,8,9,...,8790,8791,8792,8793,8794,8795,8796,8797,8798,8799
num_label0,22,26,120,102,120,74,83,91,165,169,...,117,53,117,53,73,117,53,117,73,73
num_label1,22,26,120,102,120,74,84,91,165,169,...,117,53,117,53,73,119,53,117,73,73
num_label2,22,26,120,102,120,74,84,91,165,169,...,117,53,117,53,73,117,53,117,73,73
num_label3,22,26,120,102,120,74,84,91,165,169,...,117,53,117,53,73,117,53,117,73,73
num_label4,22,26,120,102,120,74,84,91,165,169,...,117,53,117,53,73,117,53,117,73,73


In [30]:
# 求得投票众数
df_mode = df_all_transpose.mode().transpose()
df_mode.head()

,0,1,2,3,4
0,22.0,NaN,NaN,NaN,NaN
1,26.0,NaN,NaN,NaN,NaN
2,120.0,NaN,NaN,NaN,NaN
3,102.0,NaN,NaN,NaN,NaN
4,120.0,NaN,NaN,NaN,NaN


In [ ]:
# 把投票结果的数字化标签转成字符串标签
voting_class = []
for each in df_mode[0]:
  voting_class.append(num_to_class[each])
voting_class

In [32]:
# 将投票结果的字符串标签添加到df_all中
df_all['label'] = voting_class
df_all.head()

,image,num_label0,num_label1,num_label2,num_label3,num_label4,label
0,images/18353.jpg,22,22,22,22,22,asimina_triloba
1,images/18354.jpg,26,26,26,26,26,betula_nigra
2,images/18355.jpg,120,120,120,120,120,platanus_acerifolia
3,images/18356.jpg,102,102,102,102,102,pinus_bungeana
4,images/18357.jpg,120,120,120,120,120,platanus_acerifolia


In [33]:
# 提取image和label两列为最终的结果
df_submission = df_all[['image','label']].copy()
df_submission.head()

,image,label
0,images/18353.jpg,asimina_triloba
1,images/18354.jpg,betula_nigra
2,images/18355.jpg,platanus_acerifolia
3,images/18356.jpg,pinus_bungeana
4,images/18357.jpg,platanus_acerifolia


In [34]:
# 保存当前模型得到的最终结果
df_submission.to_csv('./submission-resnest.csv', index=False)
print('Voting results of resnest successfully saved!')

Voting results of resnest successfully saved!
